In [1]:
# Validation of the steady-state simulation - Case 1: MLP with Optimal settings
## Optimal Settings are like below:
# 1. Train/Validation/Test dataset ratio = 0.7/0.2/0.1
# 2. Cd scaling -> replaced as normalization for both Cl and Cd
# 3. Seperate the ML models into the two models, a model only for Cl and the other for Cd
import os
import sys
import numpy as np
import pandas as pd
import tensorflow as tf
import re
import matplotlib.pyplot as plt
from scipy import interpolate
from sklearn.model_selection import train_test_split

import datetime
%matplotlib inline

In [2]:
### Defining parameters and hyperparameters of the model

n_units=128 # Number of units in the hidden layer of the MLP network
input_size = 110 + 3 # Size of input for the network (110 coefficients and 3 other parameters, time, h, beta)
lr = 1e-03 # Learning rate of the network
test_rate = 0.1 # Defines the ratio of test dataset
val_rate = 0.2 # Defines the ratio of validation dataset
n_data = 16 # Number of txt files from which the aerodynamic coefficients are extracted
batch_size = 1000 # Mini-batch size

In [3]:
# Changing working directory

main_directory = 'D:\\VAWT_data\\flap_steady\\flap_steady'
os.chdir(main_directory)

In [4]:
## Basic parameters

c = 1 # Chord length
h = np.array([0.01, 0.02, 0.03]) * c # Height of the Gurney flaps
thickness = 0.02 * h # Thickness of the Gurney flaps
beta = np.linspace(30, 90, 5).reshape((5,1))

In [5]:
h = h.reshape((-1,1))
thickness = thickness.reshape((-1,1))

In [6]:
t_interval = 0.001
t_len = int((15-10) / t_interval)

n_beta = len(beta)# Number of the Gurney flap inclination
n_h = len(h) # Number of the height of the Gurney flaps
n_cases = n_data * t_len # Total number of cases(Number of geometries * Number of angles of attack)

In [7]:
## Creating Input dataset
# Defining time as input

main_directory = 'D:\\VAWT_data'
cm_dir = main_directory + "\\blade_1_cm_data"
cm_list = os.listdir(cm_dir)
os.chdir(cm_dir)

In [8]:
cm_target = [file for file in cm_list if file.endswith('.csv')]
cm_target = sorted(cm_target, key=lambda s: int(re.search(r'\d+',s).group()))
cm_target = [cm_target[-4],] + cm_target[:-4] + cm_target[-3:]

In [9]:
### Create input and output data
### This function is the main framework where data are reordered with respect to the shape the NNs require.
### Each input features are made by calling the corresponding functions, which generate the data,
### e.g., time, cm, h, beta, coordinates of airfoil and Gurney flaps, etc.
def genereate_input_output(cm_target, n_beta, t_len, normalize:bool=False):
    
    input_time_cm = time_and_cm(cm_target)
    t = input_time_cm[:,0].reshape((-1, 1))
    cm = input_time_cm[:,1].reshape((-1, 1))
    
    hh = generate_h(n_beta, t_len, normalize)
    bb = generate_beta(n_beta, t_len, normalize)
    total_coords = generate_coordinates(n_cases)
    
    # Concatenate data for input dataset
    x = np.hstack((t, hh, bb, total_coords))
    
    # Generating output dataset (depending on whether the data be normalized or not)
    if normalize==True:
        y = (cm-np.min(cm))/(np.max(cm)-np.min(cm))
    else:
        y = cm
    print("Dimension - x: ", x.shape)
    print("Dimension - y: ", y.shape)
    
    return x, y, t, cm

In [10]:
## Creating time for input, and Cm (moment coefficient) for output
def time_and_cm(cm_target):
    cm_df = pd.DataFrame()
    for i, file in enumerate(cm_target):
        df = pd.DataFrame()
        
        data = pd.read_csv(file, header=None)
        df = pd.concat([df, data], axis=0)
        
        time = df.iloc[:,0].values
        cm = df.iloc[:,1].values
        
        time_beUsed = time[np.where(np.logical_and(time>=10, time<15))]
        cm_beUsed = cm[np.where(np.logical_and(time>=10, time<15))]
        
        # Handle the time that is duplicated because of digits
        # Also, outliers are regulated at the second conditional statement.
        time_beUsed = handler_time(time_beUsed)
        cm_beUsed = handler_cm(cm_beUsed)
        
        linear_func = interpolate.interp1d(time_beUsed, cm_beUsed,
                                           bounds_error=False,kind='quadratic',
                                           fill_value='extrapolate')
        time_interp = np.arange(10, 15, t_interval).reshape((-1,1))
        cm_interp=linear_func(time_interp).reshape((-1,1))
        
        cm_df = pd.concat([cm_df, pd.DataFrame(np.hstack((time_interp, cm_interp)))], axis=0)
    
    input_time_cm = cm_df.iloc[:,:].values
    print("Dimension - time and $C_m$: ", input_time_cm.shape)
    return input_time_cm

In [11]:
# Handling duplicated time value
def handler_time(time_beUsed):
    for i in range(len(time_beUsed)):
        if time_beUsed[i]==time_beUsed[i-1]:
            time_beUsed[i] += 0.0005
            
    return time_beUsed

In [12]:
# Handling outlier, (if there are)
def handler_cm(cm_beUsed):
    period = int(len(cm_beUsed) / 5)
    for i in range(len(cm_beUsed)):
        if np.abs(cm_beUsed[i]-cm_beUsed[i-1])>10:
            cm_beUsed[i-1] = cm_beUsed[i-1 + period]
            
    return cm_beUsed

In [13]:
## Defining beta, the Gurney flap inclination
## In case of mere NACA0018, the bb in those indexes are considered as zero.
def generate_beta(n_beta=n_beta, t_len=t_len, normalize=True):

    beta_0 = np.zeros((t_len,1)) # Values for sheer NACA0018
    b_ = np.ones((t_len,1)) # Template for the inclination for a single h and single beta
    bb_imp = np.zeros((t_len*n_beta,1))

    for j in range(n_beta):
        b_imp = b_ * beta[j]
        bb_imp[t_len*j:t_len*(j+1),:] = b_imp[:,:]

    bb_imp = bb_imp.reshape((-1,1))
    bb = np.vstack((beta_0, bb_imp, bb_imp, bb_imp))
    if normalize==True:
        bb = bb / np.max(beta)
    
    print("Dimension - inclination(beta): ", bb.shape)

    return bb

In [14]:
## Defining the Gurney flap height
## In case of mere NACA0018, the hh in those indexes are considered as zero.
def generate_h(n_beta=n_beta, t_len=t_len, normalize:bool=True):
    hh = np.concatenate((np.zeros(t_len), h[0]*np.ones(n_beta*t_len), h[1]*np.ones(n_beta*t_len), h[2]*np.ones(n_beta*t_len)))
    hh = hh.reshape((-1,1))
    
    if normalize==True:
        hh = hh / np.max(h)
    
    print("Dimension - heights of Gurney flaps: ", hh.shape)
    return hh

In [15]:
## Generates coordinate data of NACA0018 airfoil and Gurney flaps
def generate_coordinates(n_cases):
    origin_coord = "D:\\AirfoilClCdCoordinates_out\\AirfoilClCdCoordinates_out\\airfoil15"

    csv_file_name = origin_coord + '\\airfoilOut15.txt'
    data = pd.read_csv(csv_file_name, header=None)
    
    baseline_coord_high = data.iloc[0,:]
    baseline_coord_low = data.iloc[1,:]
    baseline_coord = np.hstack((np.flip(baseline_coord_high), baseline_coord_low)).reshape((1,-1))
    airfoil_coord = np.repeat(baseline_coord, n_cases, axis=0)
    print("Dimension - airfoil coordinates: ", airfoil_coord.shape)
    
    flap_coords= coord_with_flaps(n_cases)
    total_coords = np.hstack((airfoil_coord, flap_coords))
    
    print("Dimension - total coordinates: ", total_coords.shape)
    
    return total_coords

In [16]:
# Generating data of Gurney flap coordinates
def coord_with_flaps(n_cases):
    flap_left = np.zeros((15,5))
    flap_right = np.zeros((15,5))

    for i in range(n_h):
        # Defining coordinates of the flaps with respect to beta=90 degree.
        yLeft = np.linspace(-h[i]/5, -h[i], 5).reshape((-1,1))
        yRight = np.linspace(-h[i]/5, -h[i], 5).reshape((-1,1))
        xLeft = 0.5*np.ones((5,1)) - 0.02*h[i]
        xRight = 0.5*np.ones((5,1))

        for j in range(n_beta):
            betaValue = beta[j]

            # Rotating transformation
            rotateTransf = np.array([[np.cos(90-betaValue), -np.sin(90-betaValue)],
                                     [np.sin(90-betaValue), np.cos(90-betaValue)]])
            rotateTransf = rotateTransf.reshape((2,2))

            LeftImp = np.hstack((xLeft-0.5, yLeft))
            RightImp = np.hstack((xRight-0.5, yRight))

            rotatedFlapLeft = rotateTransf @ LeftImp.T # shape: 2*5 (x-coordinates on first row, y-coordinates on second row)
            rotatedFlapRight = rotateTransf @ RightImp.T

            # All we need is the y-coordinates of the flaps
            flap_left[5*i+j,:] = rotatedFlapLeft[1,:]
            flap_right[5*i+j,:] = rotatedFlapRight[1,:]
    
    flap_coords = np.hstack((flap_left, np.flip(flap_right, axis=1)))
    flap_coords2 = np.zeros((n_cases, 10))
    
    for i in range(t_len, n_cases):
        flap_coords2[i,:] = flap_coords[i%15,:]
    print("Dimension - coord with flaps: ", flap_coords2.shape)
    
    return flap_coords2

In [17]:
# Generating x, y and cm (for denormalizing)
x, y, t, cm = genereate_input_output(cm_target, n_beta, t_len, normalize=True)

Dimension - time and $C_m$:  (80000, 2)
Dimension - heights of Gurney flaps:  (80000, 1)
Dimension - inclination(beta):  (80000, 1)
Dimension - airfoil coordinates:  (80000, 100)
Dimension - coord with flaps:  (80000, 10)
Dimension - total coordinates:  (80000, 110)
Dimension - x:  (80000, 113)
Dimension - y:  (80000, 1)


In [18]:
def dataset_split(x, y, test_rate, random_state=1, **kwargs):
    if kwargs.get('validation')==True:
        val_rate = kwargs.get('val_rate')
        x_all, x_test, y_all, y_test = train_test_split(x, y, test_size=test_rate, random_state=kwargs.get('random_state'))
        x_train, x_val, y_train, y_val = train_test_split(x_all, y_all, test_size=val_rate/(1-test_rate),  random_state=kwargs.get('random_state'))
        return x_train, x_val, x_test, y_train, y_val, y_test
    else:
        x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=test_rate, random_state=kwargs.get('random_state'))
        return x_train, x_test, y_train, y_test

In [19]:
x_train, x_val, x_test, y_train, y_val, y_test = dataset_split(x, y, test_rate, val_rate=val_rate, validation=True, random_state=26)

In [20]:
x_train.shape

(55999, 113)

In [21]:
x_train[0,:]

array([ 1.29310000e+01,  3.33333333e-01,  1.00000000e+00,  4.27163750e-03,
        8.42233786e-03,  1.24698102e-02,  1.64215642e-02,  2.02769784e-02,
        2.40405173e-02,  2.77159894e-02,  3.13048084e-02,  3.48028987e-02,
        3.82178940e-02,  4.15541698e-02,  4.48033660e-02,  4.79746630e-02,
        5.10596164e-02,  5.40608993e-02,  5.69839150e-02,  5.98111069e-02,
        6.25547652e-02,  6.52095713e-02,  6.77573919e-02,  7.02132162e-02,
        7.25691930e-02,  7.48052747e-02,  7.69256294e-02,  7.89274580e-02,
        8.08033541e-02,  8.25249766e-02,  8.41042116e-02,  8.55329314e-02,
        8.67823675e-02,  8.78450936e-02,  8.87219878e-02,  8.93836883e-02,
        8.98055634e-02,  8.99969955e-02,  8.99055367e-02,  8.95254756e-02,
        8.88320811e-02,  8.77731379e-02,  8.63376094e-02,  8.44669978e-02,
        8.20983040e-02,  7.91920393e-02,  7.56532077e-02,  7.13421891e-02,
        6.61051824e-02,  5.97000072e-02,  5.17594768e-02,  4.14328713e-02,
        2.68996353e-02, -

In [22]:
def make_mlp_model(num_layer:int = 5): # This function can only generate model with at least 3 hidden layers
    input_data = tf.keras.Input(shape=(x_train.shape[1]))

    # The first hidden layer
    x_fc = tf.keras.layers.Dense(units=n_units, activation='relu', name='fc1')(input_data)
    
    # The other hidden layers, which will be placed between the first hidden layer and the last hidden layer.
    # The number of layers that the user desires is input of this function.
    for i in range(0, num_layer-2):
        x_fc = tf.keras.layers.Dense(units=n_units, activation='relu', name='fc%d' % (i+2))(x_fc)
    
    # The last hidden layer
    x_fc_final = tf.keras.layers.Dense(units=n_units, activation='relu', name='fc%d' % num_layer)(x_fc)

    # The output layer
    output_data = tf.keras.layers.Dense(units=1, activation='linear', name='outputLayer')(x_fc_final)
    
    # MLP(FC layer)-based
    model = tf.keras.Model(input_data, output_data)
    
    return model

In [23]:
model = make_mlp_model(num_layer=5)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 113)]             0         
                                                                 
 fc1 (Dense)                 (None, 128)               14592     
                                                                 
 fc2 (Dense)                 (None, 128)               16512     
                                                                 
 fc3 (Dense)                 (None, 128)               16512     
                                                                 
 fc4 (Dense)                 (None, 128)               16512     
                                                                 
 fc5 (Dense)                 (None, 128)               16512     
                                                                 
 outputLayer (Dense)         (None, 1)                 129   

In [24]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr),
              loss = tf.keras.losses.MeanSquaredError(),
              metrics=[tf.keras.metrics.RootMeanSquaredError(name="rmse")])

In [25]:
model_directory = "D:\\TrainedModels\\20221216"
if not os.path.exists(model_directory):
    os.makedirs(model_directory)

In [26]:
ckpt_name = model_directory + "20221216unsteadyPrediction_MLP_val_"+str(val_rate) + "_test"+str(test_rate)+ "_" + str(n_units) +"units_OptimalSettings_checkpoint.h5"

ckpt = tf.keras.callbacks.ModelCheckpoint(ckpt_name, monitor="val_loss", mode='min', verbose=1, save_best_only=True)

es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=500, min_delta=1e-09,
                                      restore_best_weights=True, verbose=1)
rp = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', patience=100, factor=0.5,
                                          min_delta = 1e-10, min_lr=1e-06, verbose=1)

In [27]:
STEP_SIZE_TRAIN = len(x_train)//batch_size
VALIDATION_STEPS = len(x_val)//batch_size

In [28]:
start = datetime.datetime.now()
history = model.fit(x_train, y_train, batch_size=batch_size,
                    validation_data=(x_val, y_val),
                    steps_per_epoch = STEP_SIZE_TRAIN, validation_steps=VALIDATION_STEPS,
                    epochs=10000, shuffle=True, callbacks=[es, ckpt, rp])
end = datetime.datetime.now()

Epoch 1/10000
44/55 [=======================>......] - ETA: 0s - loss: 0.0043 - rmse: 0.0653
Epoch 1: val_loss improved from inf to 0.00001, saving model to D:\TrainedModels\2022121620221216unsteadyPrediction_MLP_val_0.2_test0.1_128units_OptimalSettings_checkpoint.h5
55/55 [==============================] - 2s 10ms/step - loss: 0.0034 - rmse: 0.0585 - val_loss: 1.0015e-05 - val_rmse: 0.0032 - lr: 0.0010
Epoch 2/10000
45/55 [=======================>......] - ETA: 0s - loss: 3.5037e-05 - rmse: 0.0059
Epoch 2: val_loss improved from 0.00001 to 0.00000, saving model to D:\TrainedModels\2022121620221216unsteadyPrediction_MLP_val_0.2_test0.1_128units_OptimalSettings_checkpoint.h5
55/55 [==============================] - 0s 5ms/step - loss: 2.9368e-05 - rmse: 0.0054 - val_loss: 3.4877e-06 - val_rmse: 0.0019 - lr: 0.0010
Epoch 3/10000
44/55 [=======================>......] - ETA: 0s - loss: 3.4257e-05 - rmse: 0.0059
Epoch 3: val_loss improved from 0.00000 to 0.00000, saving model to D:\Trained

45/55 [=======================>......] - ETA: 0s - loss: 5.3754e-06 - rmse: 0.0023
Epoch 26: val_loss did not improve from 0.00000
55/55 [==============================] - 0s 6ms/step - loss: 2.3883e-05 - rmse: 0.0049 - val_loss: 3.2718e-06 - val_rmse: 0.0018 - lr: 0.0010
Epoch 27/10000
55/55 [==============================] - ETA: 0s - loss: 3.2448e-05 - rmse: 0.0057
Epoch 27: val_loss did not improve from 0.00000
55/55 [==============================] - 0s 6ms/step - loss: 3.2448e-05 - rmse: 0.0057 - val_loss: 4.4055e-06 - val_rmse: 0.0021 - lr: 0.0010
Epoch 28/10000
55/55 [==============================] - ETA: 0s - loss: 2.0528e-05 - rmse: 0.0045
Epoch 28: val_loss did not improve from 0.00000
55/55 [==============================] - 0s 6ms/step - loss: 2.0528e-05 - rmse: 0.0045 - val_loss: 3.8779e-06 - val_rmse: 0.0020 - lr: 0.0010
Epoch 29/10000
45/55 [=======================>......] - ETA: 0s - loss: 3.4550e-05 - rmse: 0.0059
Epoch 29: val_loss did not improve from 0.00000
55/55

54/55 [============================>.] - ETA: 0s - loss: 3.2308e-05 - rmse: 0.0057
Epoch 54: val_loss did not improve from 0.00000
55/55 [==============================] - 0s 6ms/step - loss: 3.1742e-05 - rmse: 0.0056 - val_loss: 3.6425e-06 - val_rmse: 0.0019 - lr: 0.0010
Epoch 55/10000
53/55 [===========================>..] - ETA: 0s - loss: 2.3382e-05 - rmse: 0.0048
Epoch 55: val_loss did not improve from 0.00000
55/55 [==============================] - 0s 6ms/step - loss: 2.8288e-05 - rmse: 0.0053 - val_loss: 5.6491e-06 - val_rmse: 0.0024 - lr: 0.0010
Epoch 56/10000
46/55 [========================>.....] - ETA: 0s - loss: 3.3310e-05 - rmse: 0.0058
Epoch 56: val_loss did not improve from 0.00000
55/55 [==============================] - 0s 6ms/step - loss: 2.8136e-05 - rmse: 0.0053 - val_loss: 2.8199e-06 - val_rmse: 0.0017 - lr: 0.0010
Epoch 57/10000
55/55 [==============================] - ETA: 0s - loss: 2.8275e-05 - rmse: 0.0053
Epoch 57: val_loss did not improve from 0.00000
55/55

55/55 [==============================] - 0s 6ms/step - loss: 3.9864e-05 - rmse: 0.0063 - val_loss: 2.5731e-06 - val_rmse: 0.0016 - lr: 0.0010
Epoch 83/10000
46/55 [========================>.....] - ETA: 0s - loss: 3.2422e-05 - rmse: 0.0057
Epoch 83: val_loss did not improve from 0.00000
55/55 [==============================] - 0s 6ms/step - loss: 2.9687e-05 - rmse: 0.0054 - val_loss: 2.7639e-06 - val_rmse: 0.0017 - lr: 0.0010
Epoch 84/10000
46/55 [========================>.....] - ETA: 0s - loss: 9.9860e-06 - rmse: 0.0032
Epoch 84: val_loss improved from 0.00000 to 0.00000, saving model to D:\TrainedModels\2022121620221216unsteadyPrediction_MLP_val_0.2_test0.1_128units_OptimalSettings_checkpoint.h5
55/55 [==============================] - 0s 7ms/step - loss: 8.4630e-06 - rmse: 0.0029 - val_loss: 2.5684e-06 - val_rmse: 0.0016 - lr: 0.0010
Epoch 85/10000
45/55 [=======================>......] - ETA: 0s - loss: 3.9176e-05 - rmse: 0.0063
Epoch 85: val_loss did not improve from 0.00000
55/5

55/55 [==============================] - 0s 6ms/step - loss: 2.7510e-05 - rmse: 0.0052 - val_loss: 2.6473e-06 - val_rmse: 0.0016 - lr: 0.0010
Epoch 111/10000
46/55 [========================>.....] - ETA: 0s - loss: 3.2505e-05 - rmse: 0.0057
Epoch 111: val_loss did not improve from 0.00000
55/55 [==============================] - 0s 6ms/step - loss: 2.7305e-05 - rmse: 0.0052 - val_loss: 2.5740e-06 - val_rmse: 0.0016 - lr: 0.0010
Epoch 112/10000
47/55 [========================>.....] - ETA: 0s - loss: 2.5332e-05 - rmse: 0.0050
Epoch 112: val_loss did not improve from 0.00000
55/55 [==============================] - 0s 6ms/step - loss: 2.7453e-05 - rmse: 0.0052 - val_loss: 2.6628e-06 - val_rmse: 0.0016 - lr: 0.0010
Epoch 113/10000
46/55 [========================>.....] - ETA: 0s - loss: 3.2913e-05 - rmse: 0.0057
Epoch 113: val_loss did not improve from 0.00000
55/55 [==============================] - 0s 6ms/step - loss: 2.7660e-05 - rmse: 0.0053 - val_loss: 2.6305e-06 - val_rmse: 0.0016 -

...

55/55 [==============================] - 0s 6ms/step - loss: 3.3577e-05 - rmse: 0.0058 - val_loss: 2.5632e-06 - val_rmse: 0.0016 - lr: 6.2500e-05
Epoch 830/10000
45/55 [=======================>......] - ETA: 0s - loss: 2.4827e-05 - rmse: 0.0050
Epoch 830: val_loss did not improve from 0.00000
55/55 [==============================] - 0s 6ms/step - loss: 3.8556e-05 - rmse: 0.0062 - val_loss: 2.5673e-06 - val_rmse: 0.0016 - lr: 6.2500e-05
Epoch 831/10000
45/55 [=======================>......] - ETA: 0s - loss: 1.0705e-05 - rmse: 0.0033
Epoch 831: val_loss did not improve from 0.00000
55/55 [==============================] - 0s 6ms/step - loss: 8.8819e-06 - rmse: 0.0030 - val_loss: 2.5596e-06 - val_rmse: 0.0016 - lr: 6.2500e-05
Epoch 832/10000
46/55 [========================>.....] - ETA: 0s - loss: 3.2037e-05 - rmse: 0.0057
Epoch 832: val_loss did not improve from 0.00000
55/55 [==============================] - 0s 6ms/step - loss: 2.7006e-05 - rmse: 0.0052 - val_loss: 2.5631e-06 - val_rm

55/55 [==============================] - 0s 6ms/step - loss: 2.9138e-05 - rmse: 0.0054 - val_loss: 2.5877e-06 - val_rmse: 0.0016 - lr: 6.2500e-05
Epoch 858/10000
45/55 [=======================>......] - ETA: 0s - loss: 3.0250e-05 - rmse: 0.0055
Epoch 858: val_loss did not improve from 0.00000
55/55 [==============================] - 0s 6ms/step - loss: 2.4873e-05 - rmse: 0.0050 - val_loss: 2.5629e-06 - val_rmse: 0.0016 - lr: 6.2500e-05
Epoch 859/10000
55/55 [==============================] - ETA: 0s - loss: 4.2680e-06 - rmse: 0.0021
Epoch 859: val_loss did not improve from 0.00000
55/55 [==============================] - 0s 6ms/step - loss: 4.2680e-06 - rmse: 0.0021 - val_loss: 2.5601e-06 - val_rmse: 0.0016 - lr: 6.2500e-05
Epoch 860/10000
47/55 [========================>.....] - ETA: 0s - loss: 3.8249e-05 - rmse: 0.0062
Epoch 860: val_loss did not improve from 0.00000
55/55 [==============================] - 0s 6ms/step - loss: 3.2783e-05 - rmse: 0.0057 - val_loss: 2.5602e-06 - val_rm

55/55 [==============================] - 0s 7ms/step - loss: 8.8799e-06 - rmse: 0.0030 - val_loss: 2.5597e-06 - val_rmse: 0.0016 - lr: 6.2500e-05
Epoch 886/10000
48/55 [=========================>....] - ETA: 0s - loss: 3.0798e-05 - rmse: 0.0055
Epoch 886: val_loss did not improve from 0.00000
55/55 [==============================] - 0s 7ms/step - loss: 2.7183e-05 - rmse: 0.0052 - val_loss: 2.5627e-06 - val_rmse: 0.0016 - lr: 6.2500e-05
Epoch 887/10000
47/55 [========================>.....] - ETA: 0s - loss: 3.1037e-05 - rmse: 0.0056
Epoch 887: val_loss did not improve from 0.00000
55/55 [==============================] - 0s 7ms/step - loss: 2.7520e-05 - rmse: 0.0052 - val_loss: 2.5617e-06 - val_rmse: 0.0016 - lr: 6.2500e-05
Epoch 888/10000
47/55 [========================>.....] - ETA: 0s - loss: 3.0437e-05 - rmse: 0.0055
Epoch 888: val_loss did not improve from 0.00000
55/55 [==============================] - 0s 7ms/step - loss: 3.1783e-05 - rmse: 0.0056 - val_loss: 2.5632e-06 - val_rm

54/55 [============================>.] - ETA: 0s - loss: 2.2311e-05 - rmse: 0.0047
Epoch 913: val_loss did not improve from 0.00000
55/55 [==============================] - 0s 5ms/step - loss: 2.1916e-05 - rmse: 0.0047 - val_loss: 2.5601e-06 - val_rmse: 0.0016 - lr: 3.1250e-05
Epoch 914/10000
43/55 [======================>.......] - ETA: 0s - loss: 1.7678e-05 - rmse: 0.0042  
Epoch 914: val_loss did not improve from 0.00000
55/55 [==============================] - 4s 5ms/step - loss: 1.3966e-05 - rmse: 0.0037 - val_loss: 2.5602e-06 - val_rmse: 0.0016 - lr: 3.1250e-05
Epoch 915/10000
55/55 [==============================] - ETA: 0s - loss: 4.5930e-05 - rmse: 0.0068
Epoch 915: val_loss did not improve from 0.00000
55/55 [==============================] - 0s 5ms/step - loss: 4.5930e-05 - rmse: 0.0068 - val_loss: 2.5603e-06 - val_rmse: 0.0016 - lr: 3.1250e-05
Epoch 916/10000
55/55 [==============================] - ETA: 0s - loss: 2.1324e-05 - rmse: 0.0046
Epoch 916: val_loss did not impro

KeyboardInterrupt: 

In [ ]:
time = end - start
print("Training time:", time)

In [ ]:
storage_dir = "D:\\VAWT_data\\flap_unsteady\\result\\"+"20221216MLP_optimalSettings\\test"+str(test_rate)+"_val"+str(val_rate)+"_CmPrediction"
if not os.path.exists(storage_dir):
    os.makedirs(storage_dir)
os.chdir(storage_dir)

In [ ]:
hist = history.history
plt.plot(hist['loss'], lw=2, label='Training loss')
plt.plot(hist['val_loss'], lw=2, label='Validation loss')
plt.title('Training loss (mean squared error)\nMLP, optimal settings, $C_m$ prediction', size=15)
plt.xlabel('Epoch', size=15)
plt.yscale('log')
#plt.ylim([5e-5, 1e-1])
plt.tick_params(axis='both', which='major', labelsize=15)
plt.grid()
plt.legend(loc='upper right')
saveName = "TrainingLoss_test"+str(test_rate) + ".jpg"
plt.savefig(saveName, dpi=300, bbox_inches='tight', pad_inches=0.3)
plt.show()

In [ ]:
plt.plot(hist['rmse'], lw=2)
plt.plot(hist['val_rmse'], lw=2)
plt.title('Root Mean Squared Error, optimal settings\n$C_m$ prediction', size=15)
plt.xlabel('Epoch', size=15)
plt.tick_params(axis='both', which='major', labelsize=15)
plt.show()

In [ ]:
plt.plot(hist['rmse'], lw=2, label='Training RMSE')
plt.plot(hist['val_loss'], lw=2, label='Validation RMSE')
plt.title('Root Mean Squared Error\nMLP, optimal settings\n$C_m$ prediction', size=15)
plt.xlabel('Epoch', size=15)
plt.yscale('log')
plt.tick_params(axis='both', which='major', labelsize=15)
plt.grid()
plt.legend(loc='upper right')
saveName = "RMSE_test"+str(test_rate) + ".jpg"
plt.savefig(saveName, dpi=300, bbox_inches='tight', pad_inches=0.3)
plt.show()

In [ ]:
train_results = model.evaluate(x_train, y_train)

In [ ]:
val_results = model.evaluate(x_val, y_val)

In [ ]:
test_results = model.evaluate(x_test, y_test)

In [ ]:
decoded_train_ = model.predict(x_train)
decoded_val_ = model.predict(x_val)
decoded_test_ = model.predict(x_test)

In [ ]:
def define_index(cm, y):
    return np.unique(np.where(np.isin(cm, y)))

In [ ]:
def error_absolute(y_pred, y_true):
    return np.abs(y_pred - y_true)

In [ ]:
# Denormalize 
def denormalize(y):
    return y*(np.max(cm)-np.min(cm))+np.min(cm)

In [ ]:
index_train = define_index(y, y_train)
index_val = define_index(y, y_val)
index_test = define_index(y, y_test)

In [ ]:
y_train = denormalize(y_train)
y_val = denormalize(y_val)
y_test = denormalize(y_test)

In [ ]:
decoded_train = denormalize(decoded_train_)
decoded_val = denormalize(decoded_val_)
decoded_test = denormalize(decoded_test_)

In [ ]:
model_directory = "D:\\TrainedModels\\20221216"
if not os.path.exists(model_directory):
    os.makedirs(model_directory)
os.chdir(model_directory)
model_name = "20221216unsteadyValidation_MLP_val_"+str(val_rate)+"_test"+str(test_rate)+ "_" + str(n_units) +"units_optimalSettings_CmPrediction.h5"
model.save(model_name, overwrite=True, include_optimizer=True, save_format='h5')

In [ ]:
error_train_abs = error_absolute(decoded_train, y_train)
error_val_abs = error_absolute(decoded_val, y_val)
error_test_abs = error_absolute(decoded_test, y_test)

In [ ]:
os.chdir(storage_dir)

In [ ]:
def l2_error(y_pred, y_real):
    return np.sqrt(np.sum((y_pred - y_real)**2) / np.sum(y_real**2))

def mape(y_pred, y_real):
    return 100/len(y_real)*np.sum(np.abs((y_real-y_pred)/y_real))

def smape(y_pred, y_real):
    return 100*np.sum(np.abs(y_pred-y_real))/np.sum(y_real+y_pred)

In [ ]:
l2_error_train = l2_error(decoded_train, y_train)
mape_train = mape(decoded_train, y_train)
print(l2_error_train)
print(mape_train)

In [ ]:
l2_error_val = l2_error(decoded_val, y_val)
mape_val= mape(decoded_val, y_val)
print(l2_error_val)
print(mape_val)

In [ ]:
l2_error_test = l2_error(decoded_test, y_test)
mape_test= mape(decoded_test, y_test)
print(l2_error_test)
print(mape_test)

In [ ]:
l2_error_train_list = []
for i in range(0, len(x_train)):
    l2_error_train_data = l2_error(decoded_train[i], y_train[i])
    l2_error_train_list.append(l2_error_train_data)

In [ ]:
l2_error_val_list = []
for i in range(0, len(x_val)):
    l2_error_val_data = l2_error(decoded_val[i], y_val[i])
    l2_error_val_list.append(l2_error_val_data)

In [ ]:
l2_error_test_list = []
for i in range(0, len(x_test)):
    l2_error_test_data = l2_error(decoded_test[i], y_test[i])
    l2_error_test_list.append(l2_error_test_data)

In [ ]:
mape_train_list = []
for i in range(0, len(x_train)):
    mape_train_data = mape(decoded_train[i], y_train[i])
    mape_train_list.append(mape_train_data)

In [ ]:
mape_val_list = []
for i in range(0, len(x_val)):
    mape_val_data = mape(decoded_val[i], y_val[i])
    mape_val_list.append(mape_val_data)

In [ ]:
mape_test_list = []
for i in range(0, len(x_test)):
    mape_test_data = mape(decoded_test[i], y_test[i])
    mape_test_list.append(mape_test_data)

In [ ]:
def distribution_plot(y_pred, y_real, dataset="train"):
    dictionary_name = {"train":"training", "val":"validation", "test":"test"}
    dictionary_data = {"train":l2_error_train_list, "val":l2_error_val_list, "test":l2_error_test_list}
    plot_title = '$L_2$ error norm distribution - MLP, unsteady, '+ dictionary_name.get(dataset) +'.\nValidation rate {0}, test rate {1}, optimal settings ($C_m$)'.format(
        val_rate, test_rate)
    plt.plot(np.linspace(1,y_real.shape[0],y_real.shape[0]),
         l2_error_train*np.ones(y_real.shape[0],), 'k', lw=2.5)
    plt.scatter(np.linspace(1, y_real.shape[0], y_real.shape[0]), dictionary_data.get(dataset), c='b')
    plt.xlabel('Index', fontsize=15)
    plt.ylabel('$L_2$ error norm', fontsize=15)
    plt.yscale('log')
    plt.title(plot_title, fontsize=15)
    plt.grid()

In [ ]:
distribution_plot(decoded_train, y_train, dataset="train")
saveName = "trainingErrorDistribution.jpg"
plt.savefig(saveName, dpi=300, bbox_inches='tight', pad_inches=0.3)
plt.show()

In [ ]:
distribution_plot(decoded_val, y_val, dataset="val")
saveName = "validationErrorDistribution.jpg"
plt.savefig(saveName, dpi=300, bbox_inches='tight', pad_inches=0.3)
plt.show()

In [ ]:
distribution_plot(decoded_test, y_test, dataset="test")
saveName = "testErrorDistribution.jpg"
plt.savefig(saveName, dpi=300, bbox_inches='tight', pad_inches=0.3)
plt.show()

In [ ]:
predicted = model.predict(x)

In [ ]:
np.max(x)

In [ ]:
t_train = t[index_train]
t_val = t[index_val]
t_test = t[index_test]

In [ ]:
iTrain=[]
iVal=[]
iTest=[]
for i, index in enumerate(index_train):
    iTrain.append(y[index])
for k , index in enumerate(index_val):
    iVal.append(y[index])
for j, index in enumerate(index_test):
    iTest.append(y[index])

In [ ]:
iTrain = np.array(iTrain)
iVal = np.array(iVal)
iTest = np.array(iTest)

In [ ]:
def Cm_trainTestSplit_Plot(i, Cm, cm, tTrain, tTest, iTrain, iTest):
    
    title_0_Cm = 'Gurney flap not attached (NACA0018)\n$C_m$ prediction, $L_2$ error=%.4f' % l2_error_Cm    
    title_n_Cm = 'Gurney flap attached h=%.2f, '%(h[i]) + r'$\beta$=%d'%(beta[i])+'\n$C_m$ prediction, $L_2$ error=%.4f'%(l2_error_Cm)
    
    if i==0:
        title_Cm = title_0_Cm
        savename1 = "CmComparison_NACA0018.jpg"
    else:
        title_Cm = title_n_Cm
        savename1 = "CmComparison_h"+str(h[i])+"_beta"+str(beta[i])+".jpg"
    
    # CD graph plot
    plt.plot(t[:5000], Cm, 'k-', label='Ground truth')
    plt.plot(t[:5000], cm, 'k--', label='Predicted value')
    plt.scatter(tTrain, iTrain, color='b', label='Training set')
    plt.scatter(tVal, iVal, color='g', label='Validation set')
    plt.scatter(tTest, iTest, color='r', label='Test set')
    plt.xlabel('Rev.')
    plt.ylabel('$C_m$')
    plt.title(title_Cm, fontsize=15)        
    plt.legend(loc='upper left')
#     plt.ylim([0, 0.12])
    plt.grid()
    plt.savefig(savename1, dpi=300, bbox_inches='tight', pad_inches=0.3)
    plt.show()

In [ ]:
h = [0, 0.01, 0.01, 0.02, 0.02, 0.03, 0.03, 0.01, 0.02, 0.03, 0.01, 0.02, 0.03, 0.01, 0.02, 0.03]
beta = [0, 30, 45, 30, 45, 30, 45, 90, 90, 90, 60, 60, 75, 75, 75]
for i in range(0, 16):
    # Index from each dataset
    iTrain = []
    iVal = []
    iTest = []
    
    # Index from input data (alpha, in this case)
    t_train = []
    t_val = []
    t_test = []
    
    predictedValue = predicted[t_len*i:t_len*(i+1),:]
    y_corres = y[t_len*i:t_len*(i+1),:]
    
    l2_error_Cm = np.sqrt(np.sum((predictedValue - y_corres)**2) / np.sum(y_corres**2))
    
    if i==0:
        print('NACA0018 airfoil without Gurney flap\nL2 error of Cm: {0:0.4f}'.format(l2_error_Cm))
    else:
        print('L2 error of Cm: {0:0.4f}'.format(l2_error_Cm))
    
    cm = denormalize(predictedValue)
    Cm = denormalize(y_corres)
    
    for ii, index in enumerate(index_train[np.where((index_train<(i+1)*t_len) & ((index_train>=i*t_len)))]):
        iTrain.append(predicted[index])
    for jj, index in enumerate(index_val[np.where((index_val<(i+1)*t_len) & ((index_val>=i*t_len)))]):
        iVal.append(predicted[index])    
    for kk, index in enumerate(index_test[np.where((index_test<(i+1)*t_len) & (index_test>=i*t_len))]):
        iTest.append(predicted[index])
        
#     iTrain = denormalize(np.array(iTrain))
#     iTest = denormalize(np.array(iTest))
#     iVal = denormalize(np.array(iVal))
    iTrain = np.array(iTrain)
    iVal = np.array(iVal)
    iTest = np.array(iTest)
    
    for ii, index in enumerate(index_train[np.where((index_train<(i+1)*t_len) & ((index_train>=i*t_len)))]):
        t_train.append(t[index])
    for kk, index in enumerate(index_val[np.where((index_val<(i+1)*t_len) & ((index_val>=i*t_len)))]):
        t_val.append(t[index])
    for jj, index in enumerate(index_test[np.where((index_test<(i+1)*t_len) & ((index_test>=i*t_len)))]):
        t_test.append(t[index])
        
    tTrain = np.array(t_train)
    tVal = np.array(t_val)
    tTest = np.array(t_test)
    
    Cm_trainTestSplit_Plot(i, Cm, cm, tTrain, tTest, iTrain, iTest)

In [ ]:
y

In [ ]:
cm